In [3]:
import pandas as pd
import csv

In [4]:
df= pd.read_csv('Google Maps is testing a combined Commute tab to replace Driving and Transit.csv')
#df.head(10)

########function#####

def create_dictionary(key,values):
    node_edges_list= {}  
    #print(values)
    for x in key:
        node_edges_list[x]=values     
                      
        #print(node_edges_list)
    return(node_edges_list)

######end-funtion####

########## Bepth First search Algorithem########
def dfs(graph, node, visited=[]):
    print("graph is ", graph)
    print("node is ",node)
    print("the visited nodes are ",visited)
    visited.append(node)
    for n in graph[node]:
        if n not in visited:
              
            print("value of n is ", n)
            print("value of graph[node] is ", graph[node])
            visited= dfs(graph,n, visited)
    return visited

################# End Algorithem##############


nodes= df['comment_Text']
lista= [] #create empty list
node_edges= {} # empty dictionary for making nodes-edges pairs
final_node_edges_dic={}
#edges1= ['0','9clgh7', 'e5bqkhk']
edges1= df['parent_id'].unique() # get unique colums values
remove_top_element=list(edges1)  # convert array into list
remove_top_element.pop(0) # remove top element from list which is root node
#print(remove_top_element)
#print(edges1)
for edge in range(len(remove_top_element)): # code for making pairs of edges and nodes from the dataset
    #print(edges1[edge])
    lista.append(remove_top_element[edge])
    #print("parent_id",lista)
    edge_single=df.loc[df['parent_id'].isin(lista)] #get comments based on primary key and forign key
    edges_vertices=edge_single['comment_id'].values.tolist() # get correspoding comment_id for each vertices, which will shows possible reation between 2 vertices
    #edges_vertices_list.append(edges_vertices) #getting values from dataform and convert that into list.
    #print("comments_id",edges_vertices)
    node_edges= create_dictionary(lista,edges_vertices)
    final_node_edges_dic.update(node_edges) 
    
    lista =[]
#graph= {1:[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 2:[21], 21: [], 3:[]}
print(final_node_edges_dic) #final list of pairs
#visited = dfs(final_node_edges_dic,'9clgh7')
#print(visited)

   


{'9clgh7': ['e5bqkhk', 'e5bwn4g', 'e5btxu4', 'e5bmpya', 'e5bs4w3', 'e5c9mr9', 'e5bwzrb', 'e5d5dph', 'e5d3fyg', 'e5d237r', 'e5colkv', 'e5bumvv', 'e5bhqef', 'e5c1e4u', 'e5bs6c7', 'e5bhr9p', 'e5crv0y', 'e5bpnup', 'e5bi1ce', 'e5bh1by'], 'e5bqkhk': ['e5c4xqn'], 'e5bs6c7': ['e5c7vu0', 'e5d41nl', 'e5c08ws'], 'e5c7vu0': ['e5d85sh', 'e5cgv1o'], 'e5c08ws': ['e5c3xex'], 'e5bhr9p': ['e5choob', 'e5bixvt'], 'e5bixvt': ['e5bqzlk', 'e5bo31c', 'e5bqj1h'], 'e5bqzlk': ['e5ciyid'], 'e5bo31c': ['e5bv6i3', 'e5bwgvp'], 'e5crv0y': ['e5cv318'], 'e5bpnup': ['e5bu127'], 'e5bu127': ['e5d9oa2', 'e5c2txt'], 'e5c2txt': ['e5chm3o'], 'e5bi1ce': ['e5bs0cs', 'e5c03j5', 'e5bu0gl', 'e5bo4qs'], 'e5bs0cs': ['e5c0qe6'], 'e5c0qe6': ['e5c574t'], 'e5c574t': ['e5d7mqx'], 'e5d7mqx': ['e5d9jcp'], 'e5d9jcp': ['e5dizid'], 'e5bo4qs': ['e5bqf79'], 'e5bqf79': ['e5bv2j3'], 'e5bh1by': ['e5bhwu7', 'e5bk8f0', 'e5bhsdu'], 'e5bhwu7': ['e5c001z'], 'e5c001z': ['e5chi75', 'e5d0jfz'], 'e5d0jfz': ['e5daofw'], 'e5daofw': ['e5dbmuv'], 'e5bk8f0': ['

In [5]:
father_children = final_node_edges_dic
node_height = {}
for father in father_children:
    cur_height = 0
    if father in node_height:
        cur_height = node_height[father]
    else:
        node_height[father] = 0
    children = father_children[father]
    for child in children:
        if child not in node_height:
            node_height[child] = cur_height + 1
node_height = sorted(node_height.items(), key=lambda d:d[1], reverse=False)
O = [x[0] for x in node_height]

children_father = {}
for father in father_children:
    children = father_children[father]
    for child in children:
        children_father[child] = father

comment_ids = df["comment_id"]
tags = df["Rationale_Type"]
tags = tags.fillna(0)
id_tag = {}
for index, comment_id in comment_ids.items():
    id_tag[comment_id] = tags[index]
    
D_S_Set = {}
T_S_Set = {}
N_Set = {}

O.reverse()

for node in O:
    D_S_Set[node] = []
    T_S_Set[node] = []
    N_Set[node] = []
    if node in father_children:
        for child in father_children[node]:
            if id_tag[child] == "claim-support":
                D_S_Set[node].append(child)
            elif id_tag[child] == "claim-attack":
                N_Set[node].append(child)



ResSet = []
ignore_list = []
for N in O:
    if id_tag[N] == "issue" or id_tag[N] == "feature" or id_tag[N] == 0:
        result_Set = N_Set[N]  
        for node in ignore_list:
            if node in result_Set:
                result_Set.remove(node)
        if len(result_Set) != 0:
            ignore_list.append(N) 
            for node in D_S_Set[N]:
                ignore_list.append(node)
        result_Set = D_S_Set[N] + T_S_Set[N] + N_Set[N] + [N]
        for node in ignore_list:
            if node in result_Set:
                result_Set.remove(node)
        ResSet.append(result_Set)
    else:
        if id_tag[N][:5] == "claim":
            f = children_father[N]
            if len(N_Set[N]) != 0:
                for node in ignore_list:
                    if node in N_Set[N]:
                        N_Set[N].remove(node)
                if len(N_Set[N]) != 0:
                    ignore_list.append(N)
                    for node in D_S_Set[N]:
                        ignore_list.append(node)
            if id_tag[N][-7:] == "support":
                D_S_Set[f] = list(set(D_S_Set[f] + D_S_Set[N]))
                T_S_Set[f] = list(set(T_S_Set[f] + T_S_Set[N]))
                N_Set[f] = list(set(N_Set[f] + N_Set[N]))
            elif id_tag[N][-6:] == "attack":
                N_Set[f] = list(set(N_Set[f] + D_S_Set[N]))
                N_Set[f] = list(set(N_Set[f] + T_S_Set[N]))
                T_S_Set[f] = list(set(T_S_Set[f] + N_Set[N]))
print(ResSet)

[['e5dbn4z'], ['e5dizid'], ['e5d9jcp'], ['e5chm3o'], ['e5ciyid'], ['e5c9tbc', 'e5bo0t5'], ['e5diuxz'], ['e5c2txt'], ['e5d9oa2'], ['e5bqzlk'], ['e5bk8f0'], ['e5bqj1h', 'e5bixvt'], ['e5choob'], ['e5d41nl'], ['e5bv2j3', 'e5c0qe6', 'e5bs0cs', 'e5c03j5', 'e5bo4qs', 'e5c574t', 'e5bqf79', 'e5d7mqx'], ['e5cv318', 'e5crv0y'], ['e5bhr9p'], ['e5c3xex', 'e5c08ws', 'e5d85sh', 'e5cgv1o', 'e5bs6c7'], ['e5c1e4u'], ['e5bumvv'], ['e5d237r'], ['e5d3fyg'], ['e5c9mr9'], ['e5bmpya'], ['e5btxu4'], ['e5bwn4g'], ['e5bqkhk'], ['e5cyx3h', 'e5d0jfz', 'e5bzx5i', 'e5djgug', 'e5d6vd7', 'e5dbmuv', 'e5bu127', 'e5bhwu7', 'e5bh1by', 'e5cwhu1', 'e5bs4w3', 'e5bpnup']]
